# Neural Networks

In [1]:
# increase the width of the notebook
from IPython.display import display, HTML, Markdown

display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

## Separate features and target

In [3]:
# Load data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

y_train = train["Score"]
y_test = test["Score"]

X_train = train.drop("Score", axis=1)
X_test = test.drop("Score", axis=1)

## Transformations

In [4]:
#Preprocessing pipelines
numeric_features = ["WhiteElo", "EloDif"]
categorical_features = ["Opening_name", "Time_format", "Increment_binary"]

numeric_transformer = Pipeline([
    ("scaler", StandardScaler())
])
categorical_transformer = Pipeline([
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

In [5]:
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed  = preprocessor.transform(X_test)


In [6]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# 1) Prepare dense inputs
if hasattr(X_train_transformed, "toarray"):
    X_train_nn = X_train_transformed.toarray()
    X_test_nn  = X_test_transformed.toarray()
else:
    X_train_nn = X_train_transformed
    X_test_nn  = X_test_transformed

# 2) Encode string labels as integers, then one‑hot
le = LabelEncoder()
y_train_int = le.fit_transform(y_train)
y_test_int  = le.transform(y_test)
y_train_cat = to_categorical(y_train_int)
y_test_cat  = to_categorical(y_test_int)

# 3) Build a simple MLP
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_nn.shape[1],)),
    Dense(32, activation='relu'),
    Dense(y_train_cat.shape[1], activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 4) Train
history = model.fit(
    X_train_nn, y_train_cat,
    validation_split=0.1,
    epochs=30,
    batch_size=32,
    verbose=2
)

# 5) Evaluate on test set
test_loss, test_acc = model.evaluate(X_test_nn, y_test_cat, verbose=0)
print(f"Test accuracy (NN): {test_acc:.3f}")

# 6) Detailed classification report
y_pred_probs = model.predict(X_test_nn)
y_pred_int   = np.argmax(y_pred_probs, axis=1)
from sklearn.metrics import classification_report
print(classification_report(y_test_int, y_pred_int, target_names=le.classes_))

# 7) Save the model
model.save('simple_mlp_chess.keras')


C:\Users\dimit\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
1969/1969 - 8s - 4ms/step - accuracy: 0.5360 - loss: 0.8498 - val_accuracy: 0.5443 - val_loss: 0.8485
Epoch 2/30
1969/1969 - 3s - 2ms/step - accuracy: 0.5387 - loss: 0.8447 - val_accuracy: 0.5447 - val_loss: 0.8469
Epoch 3/30
1969/1969 - 3s - 2ms/step - accuracy: 0.5405 - loss: 0.8433 - val_accuracy: 0.5436 - val_loss: 0.8475
Epoch 4/30
1969/1969 - 3s - 2ms/step - accuracy: 0.5408 - loss: 0.8433 - val_accuracy: 0.5431 - val_loss: 0.8473
Epoch 5/30
1969/1969 - 4s - 2ms/step - accuracy: 0.5399 - loss: 0.8426 - val_accuracy: 0.5376 - val_loss: 0.8474
Epoch 6/30
1969/1969 - 3s - 2ms/step - accuracy: 0.5422 - loss: 0.8423 - val_accuracy: 0.5379 - val_loss: 0.8472
Epoch 7/30
1969/1969 - 3s - 2ms/step - accuracy: 0.5408 - loss: 0.8418 - val_accuracy: 0.5437 - val_loss: 0.8474
Epoch 8/30
1969/1969 - 3s - 2ms/step - accuracy: 0.5423 - loss: 0.8414 - val_accuracy: 0.5346 - val_loss: 0.8465
Epoch 9/30
1969/1969 - 3s - 2ms/step - accuracy: 0.5438 - loss: 0.8418 - val_accuracy: 0.5414 - 

C:\Users\dimit\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dimit\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dimit\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

   Black Win       0.54      0.46      0.50      4524
        Draw       0.00      0.00      0.00       566
   White Win       0.55      0.68      0.60      4910

    accuracy                           0.54     10000
   macro avg       0.36      0.38      0.37     10000
weighted avg       0.51      0.54      0.52     10000



### Even a neural network model was unable to exceed the 54% accuracy barrier, achieving a performance similar to our best traditional models like Random Forest, AdaBoost, and Gradient Boosting.
### This  suggests that the limitations might be more fundamental to the data itself or the inherent predictability of the task, rather than just the model architecture.

## Deeper Network

In [7]:
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam


In [8]:

# Build a deeper network
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_nn.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),

    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(y_train_cat.shape[1], activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks for early stopping + best‑model checkpointing
callbacks = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
    ModelCheckpoint('best_mlp_dropout.keras', save_best_only=True)
]

# Train
history = model.fit(
    X_train_nn, y_train_cat,
    validation_split=0.1,
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=2
)

# Evaluate
test_loss, test_acc = model.evaluate(X_test_nn, y_test_cat, verbose=0)
print("Test accuracy (deep MLP):", round(test_acc, 3))

Epoch 1/100


C:\Users\dimit\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


985/985 - 5s - 5ms/step - accuracy: 0.5011 - loss: 0.9639 - val_accuracy: 0.5370 - val_loss: 0.8488
Epoch 2/100
985/985 - 2s - 2ms/step - accuracy: 0.5343 - loss: 0.8591 - val_accuracy: 0.5313 - val_loss: 0.8495
Epoch 3/100
985/985 - 2s - 2ms/step - accuracy: 0.5350 - loss: 0.8526 - val_accuracy: 0.5357 - val_loss: 0.8471
Epoch 4/100
985/985 - 2s - 2ms/step - accuracy: 0.5371 - loss: 0.8512 - val_accuracy: 0.5403 - val_loss: 0.8484
Epoch 5/100
985/985 - 2s - 2ms/step - accuracy: 0.5352 - loss: 0.8503 - val_accuracy: 0.5450 - val_loss: 0.8468
Epoch 6/100
985/985 - 2s - 2ms/step - accuracy: 0.5348 - loss: 0.8489 - val_accuracy: 0.5443 - val_loss: 0.8462
Epoch 7/100
985/985 - 2s - 2ms/step - accuracy: 0.5363 - loss: 0.8486 - val_accuracy: 0.5413 - val_loss: 0.8476
Epoch 8/100
985/985 - 2s - 2ms/step - accuracy: 0.5363 - loss: 0.8479 - val_accuracy: 0.5441 - val_loss: 0.8484
Epoch 9/100
985/985 - 2s - 2ms/step - accuracy: 0.5348 - loss: 0.8472 - val_accuracy: 0.5373 - val_loss: 0.8468
Epoc

### By incorporating Dropout and Batch Normalization, we were able to slightly enhance our neural network's performance, reaching an accuracy of 0.544.